# Deep Learning-based for Vietnamese Topic Modeling

## PhoBERT + CNN (Convolutional Neural Network)

In [1]:
import re
import pandas as pd
import numpy as np
import tensorflow as tf
from vncorenlp import VnCoreNLP
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import classification_report
from transformers import TFAutoModel, AutoTokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [21]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [3]:
train_df = pd.read_csv("./csv/train.csv", encoding='utf-16')
test_df = pd.read_csv("./csv/test.csv", encoding='utf-16')

In [4]:
X_train = train_df["text"]
y_train = train_df[["label"]]
X_test = test_df["text"]
y_test = test_df[["label"]]

In [5]:
# Encode the labels
encoder = OneHotEncoder()
y_train_one_hot = encoder.fit_transform(y_train).toarray()
# It is possible to directly transform y_test since the label sets are the same
y_test_one_hot = encoder.transform(y_test).toarray()

y_train_one_hot

array([[1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [6]:
print(y_train.shape, y_train_one_hot.shape)
print(y_test.shape, y_test_one_hot.shape)

(33759, 1) (33759, 10)
(50373, 1) (50373, 10)


### Preprocessing on a single sample

In [7]:
# Sample text for testing
text = X_train[0]
print(text)

 Thành lập dự án POLICY phòng chống HIV/AIDS ở VN (NLĐ)- Quỹ hỗ trợ khẩn cấp về AIDS của Hoa Kỳ vừa thành lập dự án POLICY tại VN với cam kết hỗ trợ Chính phủ và nhân dân VN đối phó HIV/AIDS.Dự án có nhiệm vụ chính là cải thiện công tác phòng chống HIV/AIDS thông qua các lĩnh vực xây dựng chính sách, rà soát các văn bản pháp luật, xây dựng chiến lược quảng bá, xây dựng chương trình đào tạo về phòng chống HIV/AIDS, lên kế hoạch bố trí nguồn lực, huấn luyện và nghiên cứu về phương tiện truyền thông đại chúng, tổ chức các hoạt động nhằm giảm kỳ thị và phân biệt đối xử đối với người có HIV/AIDS... Theo TTXVN, dự án POLICY đặc biệt quan tâm đến công tác truyền thông phòng chống HIV/AIDS, coi đây là một biện pháp tích cực và hữu hiệu trong việc phòng chống có hiệu quả HIV/AIDS. Thời gian tới, dự án POLICY sẽ tiếp tục tổ chức các hoạt động nhằm nâng cao nhận thức cho những người có trách nhiệm với công tác chỉ đạo phòng chống HIV/AIDS.




In [8]:
def text_normalization(text):
    text = text.lower().strip()
    return re.sub('[!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n]', ' ', text)

In [9]:
text = text_normalization(text)
print(text)

thành lập dự án policy phòng chống hiv aids ở vn  nlđ   quỹ hỗ trợ khẩn cấp về aids của hoa kỳ vừa thành lập dự án policy tại vn với cam kết hỗ trợ chính phủ và nhân dân vn đối phó hiv aids dự án có nhiệm vụ chính là cải thiện công tác phòng chống hiv aids thông qua các lĩnh vực xây dựng chính sách  rà soát các văn bản pháp luật  xây dựng chiến lược quảng bá  xây dựng chương trình đào tạo về phòng chống hiv aids  lên kế hoạch bố trí nguồn lực  huấn luyện và nghiên cứu về phương tiện truyền thông đại chúng  tổ chức các hoạt động nhằm giảm kỳ thị và phân biệt đối xử đối với người có hiv aids    theo ttxvn  dự án policy đặc biệt quan tâm đến công tác truyền thông phòng chống hiv aids  coi đây là một biện pháp tích cực và hữu hiệu trong việc phòng chống có hiệu quả hiv aids  thời gian tới  dự án policy sẽ tiếp tục tổ chức các hoạt động nhằm nâng cao nhận thức cho những người có trách nhiệm với công tác chỉ đạo phòng chống hiv aids 


Word segmentation.

In [10]:
rdrsegmenter = VnCoreNLP("vncorenlp/VnCoreNLP-1.1.1.jar", annotators="wseg", max_heap_size='-Xmx500m')

In [11]:
sentences = rdrsegmenter.tokenize(text)
print(sentences)

[['thành_lập', 'dự_án', 'policy', 'phòng_chống', 'hiv', 'aids', 'ở', 'vn', 'nlđ', 'quỹ', 'hỗ_trợ', 'khẩn_cấp', 'về', 'aids', 'của', 'hoa_kỳ', 'vừa', 'thành_lập', 'dự_án', 'policy', 'tại', 'vn', 'với', 'cam_kết', 'hỗ_trợ', 'chính_phủ', 'và', 'nhân_dân', 'vn', 'đối_phó', 'hiv', 'aids', 'dự_án', 'có', 'nhiệm_vụ', 'chính', 'là', 'cải_thiện', 'công_tác', 'phòng_chống', 'hiv', 'aids', 'thông_qua', 'các', 'lĩnh_vực', 'xây_dựng', 'chính_sách', 'rà_soát', 'các', 'văn_bản', 'pháp_luật', 'xây_dựng', 'chiến_lược', 'quảng_bá', 'xây_dựng', 'chương_trình', 'đào_tạo', 'về', 'phòng_chống', 'hiv', 'aids', 'lên', 'kế_hoạch', 'bố_trí', 'nguồn_lực', 'huấn_luyện', 'và', 'nghiên_cứu', 'về', 'phương_tiện', 'truyền_thông', 'đại_chúng', 'tổ_chức', 'các', 'hoạt_động', 'nhằm', 'giảm', 'kỳ_thị', 'và', 'phân_biệt', 'đối_xử', 'đối_với', 'người', 'có', 'hiv', 'aids', 'theo', 'ttxvn', 'dự_án', 'policy', 'đặc_biệt', 'quan_tâm', 'đến', 'công_tác', 'truyền_thông', 'phòng_chống', 'hiv', 'aids', 'coi', 'đây', 'là', 'một', 

Testing PhoBERT in a single document.

In [12]:
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base", use_fast=False)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [13]:
phobert = TFAutoModel.from_pretrained("vinai/phobert-base")

Some layers from the model checkpoint at vinai/phobert-base were not used when initializing TFRobertaModel: ['lm_head']
- This IS expected if you are initializing TFRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFRobertaModel were initialized from the model checkpoint at vinai/phobert-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


In [14]:
phobert

In [15]:
tokenizer.encoder['thành_lập']

763

In [16]:
tokens = [tokenizer.encode(sentences[0])]
tokens = pad_sequences(tokens, maxlen=100, truncating="post", padding="post")
print(tokens[0])

[    0   763   169     3  2137     3     3    25 33756     3  1425   291
  2498    28     3     7     3   164   763   169     3    35 33756    15
  1253   291   926     6   603 33756  2971     3     3   169    10   527
   159     8  1238   247  2137     3     3   652     9   518   141   502
  1851     9   909   584   141   995  2524   141   270   643    28  2137
     3     3    72   421  1681  2684  2019     6   410    28   715  1062
  9847   116     9   132   272   197 12922     6  3043  4622   190    18
    10     3     3    63     3   169     3   234   511    30   247  1062
  2137     3     3   774]


In [17]:
document_matrix = phobert([tokens[:10]])
print(document_matrix[0])

tf.Tensor(
[[[-0.2505224  -0.35192013 -0.17607398 ...  0.03394873  0.38515067
   -0.17919447]
  [-0.14210458 -0.37149692 -0.40287185 ... -0.39811784 -0.15626681
    0.18297672]
  [-0.1855766  -0.29924142 -0.03049095 ...  0.2942223   0.22270073
    0.15344876]
  ...
  [-0.14341971 -0.03227279 -0.59179056 ... -0.07983431  0.65585667
   -0.5803716 ]
  [-0.31038564 -0.21429403 -0.5138396  ... -0.12822092  0.46492967
   -0.2527942 ]
  [-0.3902443   0.6176958  -0.07049552 ... -0.13137478  0.24289557
   -0.16345362]]], shape=(1, 100, 768), dtype=float32)


In [18]:
def text_vectorizer(dataset, maxlen=200):
    dataset = dataset.tolist()
    dataset = map(lambda x: text_normalization(x), dataset)
    sentences = [rdrsegmenter.tokenize(x) for x in dataset]
    tokens = [tokenizer.encode(sentence[0]) for sentence in sentences]
    tokens = pad_sequences(tokens, maxlen=maxlen, truncating="post", padding="post")
    vect = phobert(tokens).last_hidden_output
    return tokens

In [ ]:
X_train_vect = text_vectorizer(X_train)
X_train_vect.shape

In [ ]:
X_test_vect = text_vectorizer(X_test)
X_test_vect.shape

In [ ]:
def cnn_model(filters, maxlen=200):
  
    # Channel 1D CNN
    inp = tf.keras.Input(shape=(maxlen, 768))
    conv1 = tf.keras.layers.Conv1D(filters=filters, kernel_size=2, activation='relu')(inp)
    pool1 = tf.keras.layers.GlobalMaxPool1D()(conv1)
    conv2 = tf.keras.layers.Conv1D(filters=filters, kernel_size=3, activation='relu')(inp)
    pool2 = tf.keras.layers.GlobalMaxPool1D()(conv2)
    conv3 = tf.keras.layers.Conv1D(filters=filters, kernel_size=4, activation='relu')(inp)
    pool3 = tf.keras.layers.GlobalMaxPool1D()(conv3)
    concat = tf.concat([pool1, pool2, pool3], axis=1)
    dense1 = tf.keras.layers.Dense(filters * 2, activation='relu')(concat)
    dense2 = tf.keras.layers.Dense(filters * 2, activation='relu')(dense1)
    
    # Interpretation
    outputs = tf.keras.layers.Dense(10, activation='softmax')(dense2)
    model = tf.keras.Model(inputs=inp, outputs=outputs)
    
    # Compile
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model

In [ ]:
model = cnn_model(20)
model.summary()

In [ ]:
model.fit(
    X_train_vect,
    y_train_one_hot,
    epochs=5,
    batch_size=32
)

In [ ]:
y_pred_one_hot = model.predict(X_test_vect)

In [ ]:
y_pred = encoder.inverse_transform(y_pred_one_hot)
print(y_pred[:10])
print(y_test[:10].to_numpy())

In [ ]:
print(classification_report(y_test, y_pred, digits=4))